# AlexNet

In [1]:
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from utils import cifar10_input
%matplotlib inline

# 读取数据
这是一个包含60000张32×3232×32图片的数据库, 包含50000张训练集和10000张测试集

In [2]:
# 我们定义一个批次有64个样本
batch_size = 64

#  数据集地址
data_dir = 'E:/data/cifar10_data/cifar-10-batches-bin/'

# 获取训练集
train_imgs, train_labels = cifar10_input.inputs(eval_data=False,
                                                data_dir=data_dir,
                                                batch_size=batch_size,
                                                shuffle=True)
# 获取测试集
test_imgs, test_labels = cifar10_input.inputs(eval_data=True,
                                              data_dir=data_dir,
                                              batch_size=batch_size,
                                              shuffle=False)

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.FixedLengthRecordDataset`.
Instructions for updating:
Q

In [3]:
train_examples = cifar10_input.NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN# 训练样本的个数
test_examples = cifar10_input.NUM_EXAMPLES_PER_EPOCH_FOR_EVAL# 测试样本的个数
print('训练集样本数：%d'%train_examples)
print('测试集样本数：%d'%test_examples)

训练集样本数：50000
测试集样本数：10000


In [4]:
# 构造权重变量
def variable_weight(shape, stddev=5e-2):
    init = tf.truncated_normal_initializer(stddev=stddev)
    return tf.get_variable(initializer=init, shape=shape, name='weight')

# 构造偏置变量
def variable_bias(shape):
    init = tf.constant_initializer(0.1)
    return tf.get_variable(initializer=init, shape=shape, name='bias')

In [5]:
def conv(x, ksize, out_depth, strides, padding='SAME', act=tf.nn.relu,
         scope='conv_layer', reuse=None):
    """构造一个卷积层    
    Args:        
        x: 输入        
        ksize: 卷积核的大小, 一个⻓度为2的`list`, 例如[3, 3]        
        output_depth: 卷积核的个数        
        strides: 卷积核移动的步⻓, 一个⻓度为2的`list`, 例如[2, 2]        
        padding: 卷积核的补0策略        
        act: 完成卷积后的激活函数, 默认是`tf.nn.relu`        
        scope: 这一层的名称(可选)        
        reuse: 是否复用   
    Return:        
        out: 卷积层的结果    
    """
    # 这里默认数据是NHWC输入的
    in_depth = x.get_shape().as_list()[-1]
    
    with tf.variable_scope(scope, reuse=reuse):
        # 先构造卷积核
        kernel_shape = ksize + [in_depth, out_depth]
        with tf.variable_scope('kernel'):
            kernel = variable_weight(kernel_shape)
        
        strides = [1,strides[0], strides[1], 1]
        # 生成卷积
        conv = tf.nn.conv2d(input=x, filter=kernel, strides=strides,
                            padding=padding, name='conv')
        
        # 构造偏置
        with tf.variable_scope('bias'):
            bias = variable_bias([out_depth])
        
        # 和偏置相加
        preact = tf.nn.bias_add(conv, bias)
        
        # 添加激活层
        out = act(preact)
        
        return out

In [6]:
def max_pool(x, ksize, strides, padding='SAME', name='pool_layer'):
    """构造一个最大值池化层    
    Args:        
        x: 输入       
        ksize: pooling核的大小, 一个⻓度为2的`list`, 例如[3, 3]        
        strides: pooling核移动的步⻓, 一个⻓度为2的`list`, 例如[2, 2]        
        padding: pooling的补0策略        
        name: 这一层的名称(可选)    
    Return:        
        pooling层的结果    
    """
    return tf.nn.max_pool(value=x, ksize=[1, ksize[0], ksize[1], 1],
                          strides=[1, strides[0], strides[1], 1], padding=padding, name=name)   

In [7]:
def fc(x, out_depth, act=tf.nn.relu, scope='fully_connect', reuse=None):
    """构造一个全连接层    
    Args:        
        x: 输入        
        out_depth: 输出向量的维数        
        act: 激活函数, 默认是`tf.nn.relu`        
        scope: 名称域, 默认是`fully_connect`        
        reuse: 是否需要重用   
    """ 
    # 获取输入层通道数
    in_depth = x.get_shape().as_list()[-1]
    
    # 构造全连接层的参数
    with tf.variable_scope(scope, reuse=reuse):
        # 构造权重
        with tf.variable_scope('weight'):
            weight = variable_weight([in_depth,out_depth])
        # 构造偏置项
        with tf.variable_scope('bias'):
            bias = variable_bias([out_depth])
        # 一个线性函数
        fc = tf.nn.bias_add(tf.matmul(x, weight), bias)
        # 激活函数作用
        out = act(fc)
        return out

In [8]:
def AlexNet(inputs, reuse=None):
    """构建 Alexnet 的前向传播    
    Args:        
        inpus: 输入        
        reuse: 是否需要重用    
    Return:        
        net: alexnet的结果    
    """
    with tf.variable_scope('AlexNet', reuse=reuse):
        
        net = conv(inputs, [5,5], 64, [1,1], padding='VALID', scope='conv1')
        
        
        net = max_pool(net, [3,3], [2,2], padding='VALID', name='pool1')
        
        
        net = conv(net, [5,5], 64, [1,1], scope='conv2')
        
        
        net = max_pool(net, [3,3], [2,2], padding='VALID', name='pool2')
        
        
        net = tf.reshape(net, [-1, 6*6*64])
        
        
        net = fc(net, 384, scope='fc3')
        
        
        net = fc(net, 192, scope='fc4')
        
        
        net = fc(net, 10, scope='fc5', act=tf.identity)
        
        return net

In [9]:
train_out = AlexNet(train_imgs)
test_out = AlexNet(test_imgs, reuse=True)

# 损失函数

In [10]:
with tf.variable_scope('loss'):
    train_loss = tf.losses.sparse_softmax_cross_entropy(labels=train_labels, 
                                                        logits=train_out, scope='train')
    test_loss = tf.losses.sparse_softmax_cross_entropy(labels=test_labels,
                                                       logits=test_out, scope='test')

In [11]:
with tf.name_scope('accuracy'):
    with tf.name_scope('train'):
        train_acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(train_out, axis=-1,output_type=tf.int32),
                                                    train_labels), tf.float32))
    with tf.name_scope('train'):
        test_acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(test_out, axis=-1,output_type=tf.int32),
                                                       test_labels), tf.float32))

# 构造训练op

In [12]:
lr = 0.01
opt = tf.train.MomentumOptimizer(lr, momentum=0.9)
train_op = opt.minimize(train_loss)

In [15]:
NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN = 50000
NUM_EXAMPLES_PER_EPOCH_FOR_EVAL = 10000

BATCH_SIZE=64
def train(train_op, train_loss, train_acc, 
          val_loss, val_acc, 
          max_step, batch_size=BATCH_SIZE,
          train_log_step=1000, val_log_step=4000):
    """训练函数
      Args:
        train_op: 训练`op`
        train_loss: 训练集计算误差的`op`
        train_acc: 训练集计算正确率的`op`
        val_loss: 验证集计算误差的`op`
        val_acc: 验证集计算正确率的`op`
        max_step: 最大迭代步长
        batch_sise: 一个批次中样本的个数
        train_log_step: 每隔多少步进行一次训练集信息输出
        val_log_step: 每隔多少步进行一次验证集信息输出
    
      Return:
        None
      """
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord, sess=sess)
    
        try:
            _start = time.time()
            for step in range(max_step + 1):
                sess.run(train_op)

                if step % train_log_step == 0:
                    _end = time.time()
                    duration = _end - _start

                    train_loss_, train_acc_ = sess.run([train_loss, train_acc])

                    sec_per_batch = 1.0 * duration / train_log_step

                    print('[train]: step %d loss = %.4f acc = %.4f (%.4f / batch)' \
                      % (step, train_loss_, train_acc_, sec_per_batch))

                    _start = _end

                if step % val_log_step == 0:
                    val_loss_, val_acc_ = sess.run([val_loss, val_acc])
                    print('[val]: step %d loss = %.4f acc = %.4f' % (step, val_loss_, val_acc_))

            print('-------------------------Over all Result-------------------------')

            train_loss_, train_acc_ = _evaluation_no_bn(sess, train_loss, train_acc, NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN, batch_size)
            print('[TRAIN]: loss = %.4f acc = %.4f' % (train_loss_, train_acc_))

            val_loss_, val_acc_ = _evaluation_no_bn(sess, val_loss, val_acc, NUM_EXAMPLES_PER_EPOCH_FOR_EVAL, batch_size)
            print('[VAL]: loss = %.4f acc = %.4f' % (val_loss_, val_acc_))

        except tf.errors.OutOfRangeError:
            print('Epoch Limited. Done!')
        finally:
            coord.request_stop()
            coord.join(threads)

def _evaluation_no_bn(sess, loss_op, acc_op, num_examples, batch_size):
    max_steps = num_examples // batch_size
    losses = []
    accs = []
    for _ in range(max_steps):
        loss_value, acc_value = sess.run([loss_op, acc_op])
        losses.append(loss_value)
        accs.append(acc_value)

    mean_loss = np.array(losses, dtype=np.float32).mean()
    mean_acc = np.array(accs, dtype=np.float32).mean()

    return mean_loss, mean_acc

In [ ]:
train(train_op, train_loss, train_acc, test_loss, test_acc, 20000, batch_size)

[train]: step 0 loss = 2.3299 acc = 0.1406 (0.0083 / batch)
[val]: step 0 loss = 2.4330 acc = 0.0469
[train]: step 1000 loss = 1.1107 acc = 0.6875 (0.0402 / batch)
[train]: step 2000 loss = 0.7037 acc = 0.7344 (0.0411 / batch)
[train]: step 3000 loss = 0.8154 acc = 0.7344 (0.0437 / batch)
[train]: step 4000 loss = 0.2628 acc = 0.9062 (0.0493 / batch)
[val]: step 4000 loss = 0.8344 acc = 0.7344
[train]: step 5000 loss = 0.4574 acc = 0.8281 (0.0455 / batch)
[train]: step 6000 loss = 0.4139 acc = 0.8438 (0.0443 / batch)
[train]: step 7000 loss = 0.2401 acc = 0.9062 (0.0463 / batch)
[train]: step 8000 loss = 0.2192 acc = 0.9219 (0.0462 / batch)
[val]: step 8000 loss = 1.1674 acc = 0.7969
[train]: step 9000 loss = 0.0820 acc = 0.9844 (0.0450 / batch)
[train]: step 10000 loss = 0.2050 acc = 0.9062 (0.0449 / batch)
[train]: step 11000 loss = 0.2135 acc = 0.9219 (0.0447 / batch)
[train]: step 12000 loss = 0.1096 acc = 0.9375 (0.0454 / batch)
[val]: step 12000 loss = 1.6042 acc = 0.6094
[train]